In [1]:
# !pip install python-dotenv
# echo python-dotenv >> requirements.txt

You should consider upgrading via the '/Users/shahnam/.pyenv/versions/3.7.8/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
# !pip install psycopg2-binary

You should consider upgrading via the '/Users/shahnam/.pyenv/versions/3.7.8/bin/python3.7 -m pip install --upgrade pip' command.


In [1]:
import psycopg2
import pandas as pd
import json
import os
from dotenv import load_dotenv

In [2]:
# To check if it matches the python in docker container
!python -V

Python 3.8.8


In [3]:
pwd

'/usr/src/b2p/notebooks'

In [4]:
ls ../Data

'B2P Dataset_2020.10.xlsx'
 B2P_Rwanda_matchedIDs_final_2020-09-24.csv
 B2P_Rwanda_matchedIDs_final_2020-09-24_copy.csv
 B2P_World_Dataset_2020.01.14.xls
'Predicted Bridge Sites.xlsx'
 cleaned_merged_dataset_final_revised.csv
 main_data_clean.csv
 predict_df.csv


# Convert CSV into DataFrame format

In [5]:
#Convert CSV File W/ Data into dataframe
dfp = pd.read_csv("../Data/predict_df.csv")
df = pd.read_csv("../Data/main_data_clean.csv")

In [6]:
#Data frame looks good to go
print(df.shape, dfp.shape)
df.head()

(1472, 43) (1472, 7)


,bridge_name,bridge_opportunity_project_code,bridge_opportunity_needs_assessment,bridge_opportunity_level1_government,bridge_opportunity_level2_government,bridge_opportunity_gps_latitude,bridge_opportunity_gps_longitude,bridge_opportunity_bridge_type,bridge_opportunity_span_m,bridge_opportunity_individuals_directly_served,...,bridge_classification,flag_for_rejection,rejection_reason,bridge_type,estimated_span_m,height_differential_between_banks,bridge_opportunity_general_project_photos,bridge_opportunity_casesafeid,country,good_site
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,-2.760833,29.488056,Suspended Bridge,NaN,NaN,...,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,NaN,006f100000asFErAAM,Rwanda,0.0
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,...,NaN,NaN,NaN,NaN,10.0,0.7,https://flic.kr/s/aHskvedfPB,006f100000a82QzAAI,Rwanda,1.0
2,Karambi,1007374,NaN,Southern Province,Huye,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,...,Standard,NaN,NaN,Suspension,45.0,1.0,https://flic.kr/s/aHskbMF1Kr,006f100000a86DaAAI,Rwanda,1.0
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,...,Standard,No,NaN,Suspended,47.4,1.1,https://www.flickr.com/photos/bridgestoprosper...,006f100000a86F4AAI,Rwanda,1.0
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,...,Standard,No,NaN,Suspended,90.0,1.1,https://www.flickr.com/photos/bridgestoprosper...,006f100000a86FXAAY,Rwanda,1.0


In [7]:
dfp.columns

Index(['bridge_opportunity_bridge_type', 'bridge_opportunity_span_m',
       'days_per_year_river_is_flooded', 'bridge_classification',
       'flag_for_rejection', 'height_differential_between_banks',
       'bridge_opportunity_project_code'],
      dtype='object')

### Connect to the database in RDS DB Instance

* DB instance name is created in amazon RDS, but DB names within the DB instance are created in pgAdmin4

In [8]:
#Secrets are contained in a .envfile
file_path = os.path.abspath('../project/app/api')  # adjust as appropriate
load_dotenv(os.path.join(file_path, '.env'))
# load_dotenv()
db_name = os.getenv("DB_NAME")
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")

print(db_name)
print(db_user)
print(db_password)
print(db_host)
print(db_port)

b2p-db
b2ppostgres
postgres295
db-instance-b2p.crof5bnjgsr5.us-east-1.rds.amazonaws.com
5432


### Create tables in the database

In [9]:
#!pip install SQLAlchemy

In [12]:
import sqlalchemy  # Package for accessing SQL databases via Python
from sqlalchemy.ext.declarative import declarative_base
import logging

# Connect to database (Note: The package psychopg2 is required for Postgres to work with SQLAlchemy)
engine = sqlalchemy.create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
con = engine.connect()
# Verify that there are no existing tables
print(engine.table_names())

<ipython-input-12-9a80835f1de2>:9: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  print(engine.table_names())


['cleaneddata_table', 'model_table']


### Upload DataFrame to the DataBase tables
Only need to do it once

In [13]:
df.to_sql('cleaneddata_table', con, if_exists='replace')
dfp.to_sql('model_table', con, if_exists='replace')

In [14]:
con.close()

### Make query to the database

In [15]:
def conn_curs():
    """
    makes a connection to the database
    """
    global db_name
    global db_user
    global db_password
    global db_host
    global db_port
    
    connection = psycopg2.connect(dbname=db_name, user= db_user,
                                  password=db_password, host= db_host,port=db_port)
    cursor = connection.cursor()
    return connection, cursor

In [16]:
def fetch_query_records(query):
    global conn_curs
    conn, cursor = conn_curs()
    cursor.execute(query)
    result = cursor.fetchall()
    cursor.close()
    conn.close()
    return result

In [17]:
fetch_query_records("""SELECT bridge_name from cleaneddata_table LIMIT 5;""")

[('Bukinga',),
 ('Kagarama',),
 ('Karambi',),
 ('Rugeti',),
 ('Nyakabuye - Nkomane',)]

In [18]:
# Testing Querie to get Records based on Bridge Name
conn, cursor = conn_curs()
query  = f"""SELECT country from cleaneddata_table where bridge_name = 'Bukinga' LIMIT 1;"""
cursor.execute(query)
result = cursor.fetchall()
cursor.close()
conn.close
print(result)

[('Rwanda',)]


In [26]:
def fetch_all_records():
    global conn_curs
    conn,cursor = conn_curs()
    query = f""" SELECT * from model_table""";
    cursor.execute(query)
    result = cursor.fetchall() #when it fetches it adds an extra column (indexes), 
    # make sure to remove it
    conn.close()
    df2 = pd.DataFrame(result, columns= ['index']+list(dfp.columns))
    df2 = df2.iloc[:,1:] # This code removes extra copied indexed column
    # string json format
    df_json = df2.to_json(orient='records')
    # convert to python dictionary
    parsed = json.loads(df_json)
    return df2, parsed


In [27]:
#Testing fetch_all_records function
df2, parsed = fetch_all_records()
print(df2.shape)
df2.head()

(1472, 7)


,bridge_opportunity_bridge_type,bridge_opportunity_span_m,days_per_year_river_is_flooded,bridge_classification,flag_for_rejection,height_differential_between_banks,bridge_opportunity_project_code
0,Suspended Bridge,52.44,78.63,None,Yes,0.97,1009317
1,Suspension Bridge,48.00,120.00,None,None,0.70,1007325
2,Suspension Bridge,50.00,60.00,Standard,None,1.00,1007374
3,Suspension Bridge,68.00,60.00,Standard,No,1.10,1007466
4,Suspension Bridge,78.00,35.00,Standard,No,1.10,1007495


In [29]:
type(parsed)

list

In [31]:
parsed[0:5]

[{'bridge_opportunity_bridge_type': 'Suspended Bridge',
  'bridge_opportunity_span_m': 52.44,
  'days_per_year_river_is_flooded': 78.63,
  'bridge_classification': None,
  'flag_for_rejection': 'Yes',
  'height_differential_between_banks': 0.97,
  'bridge_opportunity_project_code': '1009317'},
 {'bridge_opportunity_bridge_type': 'Suspension Bridge',
  'bridge_opportunity_span_m': 48.0,
  'days_per_year_river_is_flooded': 120.0,
  'bridge_classification': None,
  'flag_for_rejection': None,
  'height_differential_between_banks': 0.7,
  'bridge_opportunity_project_code': '1007325'},
 {'bridge_opportunity_bridge_type': 'Suspension Bridge',
  'bridge_opportunity_span_m': 50.0,
  'days_per_year_river_is_flooded': 60.0,
  'bridge_classification': 'Standard',
  'flag_for_rejection': None,
  'height_differential_between_banks': 1.0,
  'bridge_opportunity_project_code': '1007374'},
 {'bridge_opportunity_bridge_type': 'Suspension Bridge',
  'bridge_opportunity_span_m': 68.0,
  'days_per_year_riv